## Installing Dependencies

In [1]:
%pip install ta darts xgboost

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 39.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 56.1 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 51.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 62.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 65.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.3/770.3 kB 57.0 MB/s  0:00:00
   ━━━━━━━━━━━

In [ ]:
# Replace pandas with a GPU-accelerated version backed by cuDF. this allows the use of NVIDIA GPU. 
# Don't run this if you want to use your cpu
%load_ext cudf.pandas

In [4]:
import warnings
warnings.filterwarnings("ignore", message=".*Period with BDay freq.*", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*Parameters:.*are not used.*", category=UserWarning)

# Loading Datasets

NOTE: Upload files to the colab before running the below cell

In [1]:

import pandas as pd

column_map = {'c': 'close', 'h' : 'high', 'l' : 'low', 'o' : 'open', 't' : 'date', 'v' : 'volume'}

#TO RUN on Google Cloud
df_price = pd.read_json('/content/aapl_price.json')[column_map.keys()].rename(columns=column_map)
df_fundamentals = pd.read_csv('/content/aapl_fundamentals.csv').rename(columns={'Quarter End Date' : 'date'})
df_vix = pd.read_csv('/content/vix.csv').rename(columns={'DATE' : 'date'})

#TO RUN Locally
# df_price = pd.read_json('../data/aapl_price.json')[column_map.keys()].rename(columns=column_map)
# df_fundamentals = pd.read_csv('../data/aapl_fundamentals.csv').rename(columns={'Quarter End Date' : 'date'})
# df_vix = pd.read_csv('../data/vix.csv').rename(columns={'DATE' : 'date'})
# type(df_price)

pandas.core.frame.DataFrame

In [2]:
display(df_price.head())
display(df_fundamentals.head())
display(df_vix.head())

,close,high,low,open,date,volume
0,23.78,23.78,23.02,23.16,2016-01-04T05:00:00Z,287741356
1,23.18,23.89,23.11,23.87,2016-01-05T05:00:00Z,234762144
2,22.73,23.10,22.54,22.69,2016-01-06T05:00:00Z,284319308
3,21.77,22.60,21.76,22.27,2016-01-07T05:00:00Z,343985812
4,21.88,22.37,21.84,22.24,2016-01-08T05:00:00Z,300265168


,date,EBITDA (USD millions),EV (USD millions)
0,2025-09-30,"35,550","3,790,000"
1,2025-06-30,"31,032","3,060,000"
2,2025-03-31,"32,250","3,340,000"
3,2024-12-31,"45,912","3,920,000"
4,2024-09-30,"32,502","3,550,000"


,date,OPEN,HIGH,LOW,CLOSE
0,01/02/1990,17.24,17.24,17.24,17.24
1,01/03/1990,18.19,18.19,18.19,18.19
2,01/04/1990,19.22,19.22,19.22,19.22
3,01/05/1990,20.11,20.11,20.11,20.11
4,01/08/1990,20.26,20.26,20.26,20.26


# Data Curation

### DateTime Conversion

In [ ]:
#convert the date column to be date_time type and normalized.
df_price['date'] = pd.to_datetime(df_price['date']).dt.tz_localize(None).dt.normalize()
df_fundamentals['date'] = pd.to_datetime(df_fundamentals['date']).dt.normalize()
df_vix['date'] = pd.to_datetime(df_vix['date']).dt.normalize()

In [4]:
print(df_price.dtypes)
print(df_fundamentals.dtypes)
print(df_vix.dtypes)

close            float64
high             float64
low              float64
open             float64
date      datetime64[ns]
volume             int64
dtype: object
date                     datetime64[ns]
EBITDA (USD millions)            object
EV (USD millions)                object
dtype: object
date     datetime64[ns]
OPEN            float64
HIGH            float64
LOW             float64
CLOSE           float64
dtype: object


### Numeric Column Conversion

In [ ]:
# the value is in string format. e.g 100,000.12. remove all ',', changing the type to float
df_fundamentals = df_fundamentals.replace(to_replace=',', value='', regex=True).astype({'EBITDA (USD millions)' : 'float', 'EV (USD millions)' : 'float'})

### Time Series Index

In [ ]:
# Convert all datasets to time-indexed format for chronological operations
df_price = df_price.set_index('date')
df_fundamentals = df_fundamentals.set_index('date')
df_vix = df_vix.set_index('date')

# Feature Engineering

## Normalized Close

Normalizing the closing price by its 20-day(one trading month) moving average so the model learns trend deviation instead of absolute price.  
This will capture the trend better and remove price-level bias.

In [7]:
from ta.trend import SMAIndicator

sma = SMAIndicator(df_price['close'], window=20)

df_price['norm_close'] = df_price['close']/sma.sma_indicator()

## MACD

Moving average convergence/divergence(MACD) is an indicator used to identify trend direction and momentum in stock prices.  
Generating multiple momentum-based MACD features from the raw closing price as additional features to capture trend direction and strength.


In [10]:
from ta.trend import MACD

macd = MACD(df_price['close'])

df_price['macd_line'] = macd.macd()
df_price['macd_diff'] = macd.macd_diff()
df_price['macd_signal'] = macd.macd_signal()


## RSI

Relative Strength Index (RSI) is a momentum-based indicator used to measure the speed and magnitude of recent price movements, helping identify overbought and oversold conditions in stock prices.  
Generating an RSI feature from the raw closing price as an additional feature

In [16]:
from ta.momentum import RSIIndicator

rsi = RSIIndicator(df_price['close'])

df_price['rsi'] = rsi.rsi()

## EV/EBIDTA

Enterprise Value to EBITDA (EV/EBITDA) is used to measure how expensive a company is relative to its operating earnings.  
Generating an EV/EBITDA feature from raw enterprise value and EBITDA data to capture relative company valuation.


In [18]:
df_fundamentals['ev_ebidta'] = df_fundamentals['EV (USD millions)']/df_fundamentals['EBITDA (USD millions)']

## OBV





On-Balance Volume (OBV) is a volume-based indicator used to measure buying and selling pressure by accumulating volume based on price direction.  
Generating a normalized OBV change feature by scaling daily OBV differences with its 20-day moving average to capture abnormal volume pressure.

In [11]:
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import SMAIndicator

obv = OnBalanceVolumeIndicator(df_price['close'], df_price['volume'])

obv_series = obv.on_balance_volume()

sma = SMAIndicator(obv_series, window=20)

df_price['norm_obv_diff'] = obv_series.diff() / sma.sma_indicator()

## Normalized VIX Change

The Volatility Index (VIX) measures overall market fear and uncertainty.  
Normalizing short-term VIX change feature by scaling the 5-day percentage change(one trading week) in VIX with its 20-day(one trading month) moving average to detect short-term turbulence 

In [13]:
from ta.trend import SMAIndicator

vix_sma = SMAIndicator(df_vix['CLOSE'], window=20)

df_vix['norm_vix_change'] = df_vix['CLOSE'].pct_change(5) * 100 / vix_sma.sma_indicator()

## Closing price 5 Day Returns

This column represents the percentage change in the stock price over the next 5 trading days.  
We are choosing this over daily price change as a target feature to smooth out randonmess and make the patterns more learnable.

In [14]:
df_price['price_change'] = df_price["close"].shift(-5).pct_change(periods=5) * 100

/tmp/ipykernel_37118/1659047010.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_price['price_change'] = df_price["close"].shift(-5).pct_change(periods=5) * 100


## Data Set Creation

Combining different data sets into one

In [19]:
df = pd.concat([df_price[['norm_close', 'price_change', 'macd_line', 'macd_diff', 'macd_signal', 'rsi', 'norm_obv_diff']], df_fundamentals[['ev_ebidta']], df_vix[['norm_vix_change']]], axis=1)

## Data Cleaning

Forward fill the ev_ebidta since its quarterly data

In [21]:
df['ev_ebidta'] = df['ev_ebidta'].ffill()

filtering nan rows

In [22]:
df = df.dropna()

aligning columns

In [23]:
df = df[['norm_close', 'norm_obv_diff', 'macd_line', 'macd_diff', 'macd_signal', 'rsi', 'ev_ebidta', 'norm_vix_change', 'price_change']]
df.head()

,norm_close,norm_obv_diff,macd_line,macd_diff,macd_signal,rsi,ev_ebidta,norm_vix_change,price_change
date,,,,,,,,,
2016-03-31,1.041724,-0.107127,0.559188,0.066089,0.493099,71.008470,38.940719,-0.422218,-0.404367
2016-04-01,1.047156,0.101163,0.584986,0.073510,0.511476,73.167919,38.940719,-0.717148,-1.241987
2016-04-04,1.053577,0.133250,0.618475,0.085599,0.532876,75.319795,38.940719,-0.477911,-1.864340
2016-04-05,1.037534,-0.090122,0.614530,0.065323,0.549207,68.460985,38.940719,0.757635,0.561798
2016-04-06,1.043449,0.085590,0.625177,0.060776,0.564401,71.009868,38.940719,0.259670,0.953137


writing to a csv file

In [ ]:
#TO RUN on Google Cloud
df.to_csv("/content/aapl_model_dataset.csv", index=True, header=True)

#TO RUN Locally
# df.to_csv("../data/aapl_model_dataset.csv", index=True, header=True)

# Model Implementation

## Functions

Proper time-series train/test split.  
10% of the latest time-series data will be used as test set.

In [32]:
def test_train_split(df, target_col, test_size=0.1):
  total_length = df.count().max()
  split_len = int(total_length * test_size)
  split_idx = total_length - split_len
  X, y = df[[x for x in df.columns if x != target_col]], df[[target_col]]
  X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
  y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
  return X_train, X_test, y_train, y_test, split_len

## ARIMA


Calcualte the AutoRegressive Integrated Moving Average(ARIMA) and add it as a feature column.

In [33]:
target_col = 'price_change'

### Test Train Split

In [34]:
X_train, X_test, y_train, y_test, split_len = test_train_split(df, target_col, 0.1)

### Training

In [41]:
import numpy as np
# from cuml.tsa.arima import ARIMA

# arima = ARIMA(y_train[target_col], order=(1,0,1), fit_intercept=False)
# arima.fit()


# TO RUN locally on CPU
from statsmodels.tsa.arima.model import ARIMA
arima = ARIMA(y_train[target_col], order=(1,0,1),   trend="n" )
arima_result = arima.fit()

/home/john-desktop/miniconda3/envs/rapids-new/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/john-desktop/miniconda3/envs/rapids-new/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/john-desktop/miniconda3/envs/rapids-new/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


### Prediction

In [42]:
# y_predict = pd.concat([arima.predict(end=len(y_train)), arima.forecast(split_len)], axis=0)



# TO RUN locally on CPU
y_in_sample = arima_result.predict(start=0, end=len(y_train)-1)

y_out_sample = arima_result.forecast(steps=split_len)

y_predict = pd.concat([y_in_sample, y_out_sample], axis=0)


/home/john-desktop/miniconda3/envs/rapids-new/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/john-desktop/miniconda3/envs/rapids-new/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


### Metrics

In [45]:
from sklearn.metrics import r2_score

metrics = {
    "ARIMA": {
        "Training_Accuracy": r2_score(
            y_train[target_col],
            y_predict.iloc[:len(y_train)]
        ) * 100,
        
        "Testing_Accuracy": r2_score(
            y_test[target_col],
            y_predict.iloc[len(y_train):]
        ) * 100
    }
}

print(f"Training accuracy: {metrics['ARIMA']['Training_Accuracy']:.3f}%")
print(f"Testing accuracy: {metrics['ARIMA']['Testing_Accuracy']:.3f}%")


Training accuracy: 61.162%
Testing accuracy: -0.260%


### Adding ARIMA prediction to the existing dataset

In [47]:
y_predict = y_predict.reset_index(drop=True)
y_predict.index = df.index
df['arima_pred'] = y_predict
df = df[[*[x for x in df.columns if x != target_col], target_col]]
df.head()

,norm_close,norm_obv_diff,macd_line,macd_diff,macd_signal,rsi,ev_ebidta,norm_vix_change,arima_pred,price_change
date,,,,,,,,,,
2016-03-31,1.041724,-0.107127,0.559188,0.066089,0.493099,71.008470,38.940719,-0.422218,0.000000,-0.404367
2016-04-01,1.047156,0.101163,0.584986,0.073510,0.511476,73.167919,38.940719,-0.717148,-0.318132,-1.241987
2016-04-04,1.053577,0.133250,0.618475,0.085599,0.532876,75.319795,38.940719,-0.477911,-1.009077,-1.864340
2016-04-05,1.037534,-0.090122,0.614530,0.065323,0.549207,68.460985,38.940719,0.757635,-1.477215,0.561798
2016-04-06,1.043449,0.085590,0.625177,0.060776,0.564401,71.009868,38.940719,0.259670,0.571859,0.953137


### Data Snapshot

In [ ]:
df.to_csv("/content/aapl_model_arima_dataset.csv", index=True, header=True)
#TO RUN locally
# df.to_csv("../data/aapl_model_arima_dataset.csv", index=True, header=True)

## TCN

We trained a Temporal Convolutional Network (TCN) model and added its predictions to the dataset as a new feature column.

### Train Test Split

Proper time-series train/test split.  
10% of the latest time-series data will be used as test set.

In [ ]:
from darts import TimeSeries

df = df.asfreq('B')
df = df.ffill()

feature_cols = [x for x in df.columns if x != target_col]

X = df[feature_cols]
y = df[[target_col]]

X = TimeSeries.from_dataframe(X.reset_index(), time_col='date', value_cols=feature_cols)
y = TimeSeries.from_dataframe(y.reset_index(), time_col='date', value_cols=target_col)

split_point = int(0.9 * len(y))     # 90% train
y_train, y_test = y[:split_point], y[split_point:]
X_train, X_test = X[:split_point], X[split_point:]

### Feature Scaling

Normalization is crucial for TCN to work.

In [52]:
from darts.dataprocessing.transformers.scaler import Scaler

feature_scaler = Scaler()
X_train = feature_scaler.fit_transform(X_train)
X_test = feature_scaler.transform(X_test)

target_scaler = Scaler()
y_train = target_scaler.fit_transform(y_train)
y_test = target_scaler.transform(y_test)

### Training

Train Temporal Convolutional Network (TCN) with a 60-step lookback window to perform one-step-ahead forecasting using historical covariates.

In [54]:
from darts.models import TCNModel


tcn = TCNModel(
    input_chunk_length=60,         # lookback window
    output_chunk_length=1,         # 1-day ahead
    kernel_size=3,
    num_filters=25,
    n_epochs=40,
    dropout=0.3,
    pl_trainer_kwargs={
        "accelerator": "gpu",
        "devices": [0],            # or -1 for all GPUs
    },
    optimizer_kwargs={"lr": 1e-3},
    random_state=0,
)

tcn.fit(
    series=y_train,
    past_covariates=X_train,
    verbose=False
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=40` reached.


TCNModel(output_chunk_shift=0, kernel_size=3, num_filters=25, num_layers=None, dilation_base=2, weight_norm=False, dropout=0.3, input_chunk_length=60, output_chunk_length=1, n_epochs=40, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': [0]}, optimizer_kwargs={'lr': 0.001}, random_state=0)

### Prediction

In [55]:
train_start = y_train.time_index[tcn.input_chunk_length]

y_pred_train = tcn.historical_forecasts(
    series=y_train,
    past_covariates=X_train,
    start=train_start,
    forecast_horizon=1,
    stride=1,
    retrain=False,
    last_points_only=True,
    verbose=False,
)

y_pred_test = tcn.historical_forecasts(
    series=y_test,
    past_covariates=X_test,
    forecast_horizon=1,     # predict 1 step ahead each time
    stride=1,               # every time point
    retrain=False,          # use your already-fitted model
    last_points_only=True,  # we only want the 1-step-ahead point
    verbose=False,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


### Metrics

In [57]:
from darts.metrics import r2_score

metrics['TCN'] = {
        'Training_Accuracy': r2_score(y_train, y_pred_train) * 100 ,
        'Testing_Accuracy' : r2_score(y_test, y_pred_test) * 100
        }

print(f"Training accuracy: {metrics['TCN']['Training_Accuracy']:.3f}%")
print(f"Testing accuracy: {metrics['TCN']['Testing_Accuracy']:.3f}%")


Training accuracy: 85.070%
Testing accuracy: 69.145%


### Feature Engineering

Since the TCN was trained on scaled values, the predictions must be inverse-transformed to restore their original real-world meaning.

In [58]:
y_train = target_scaler.inverse_transform(y_train)
y_test = target_scaler.inverse_transform(y_test)
y_pred_train = target_scaler.inverse_transform(y_pred_train)
y_pred_test = target_scaler.inverse_transform(y_pred_test)

#### TCN Prediction + Residual

conbining the train + test prediction result back in to one

In [59]:
df_tcn = pd.DataFrame(pd.concat([y_pred_train.to_dataframe(), y_pred_test.to_dataframe()], axis=0))
df_tcn = df_tcn.rename(columns={'price_change' : 'tcn_pred'})
df_tcn.head()


,tcn_pred
date,
2016-06-23,-0.790171
2016-06-24,2.195738
2016-06-27,4.279136
2016-06-28,2.287828
2016-06-29,2.226933


#### Add TCN predictions as a new feature

In [60]:
df = pd.concat([df, df_tcn[['tcn_pred']]], axis=1, join='inner')
df = df[[*[x for x in df.columns if x != target_col], target_col]]
df.head()

,norm_close,norm_obv_diff,macd_line,macd_diff,macd_signal,rsi,ev_ebidta,norm_vix_change,arima_pred,tcn_pred,price_change
date,,,,,,,,,,,
2016-06-23,0.982029,4.580814,-0.123874,-0.056475,-0.067399,44.465569,38.940719,-0.661015,-0.863052,-0.790171,-0.546946
2016-06-24,0.957707,14.597229,-0.173998,-0.085280,-0.088718,35.058942,38.940719,1.904920,-0.393038,2.195738,2.673546
2016-06-27,0.947827,2.487262,-0.236016,-0.117838,-0.118178,31.473744,38.940719,1.684382,2.249294,4.279136,3.284150
2016-06-28,0.967187,-1.458281,-0.253197,-0.108016,-0.145182,39.243900,38.940719,0.081447,2.568649,2.287828,2.058961
2016-06-29,0.977391,-1.103284,-0.249415,-0.083386,-0.166028,42.739961,38.940719,-1.184807,1.527974,2.226933,1.624130


#### Data Snapshot

In [ ]:
df.to_csv("/content/aapl_model_tcn_dataset.csv", index=True, header=True)
#TO RUN locally
# df.to_csv("../data/aapl_model_tcn_dataset.csv", index=True, header=True)


## TFT

We trained a Temporal Fusion Transformer (TFT) model and added its predictions to the dataset as a new feature column.

### Train Test Split

Proper time-series train/test split.  
10% of the latest time-series data will be used as test set.

In [63]:
from darts import TimeSeries

df = df.asfreq('B')
df = df.ffill()

feature_cols = [x for x in df.columns if x != target_col]

X = df[feature_cols]
y = df[[target_col]]

X = TimeSeries.from_dataframe(X.reset_index(), time_col='date', value_cols=feature_cols)
y = TimeSeries.from_dataframe(y.reset_index(), time_col='date', value_cols=target_col)

split_point = int(0.9 * len(y))     # 90% train
y_train, y_test = y[:split_point], y[split_point:]
X_train, X_test = X[:split_point], X[split_point:]

### Feature Scaling

Normalization is crucial for TFT to work.

In [64]:
from darts.dataprocessing.transformers.scaler import Scaler

feature_scaler = Scaler()
X_train = feature_scaler.fit_transform(X_train)
X_test = feature_scaler.transform(X_test)

target_scaler = Scaler()
y_train = target_scaler.fit_transform(y_train)
y_test = target_scaler.transform(y_test)

### Training

In [65]:
from darts.models import TFTModel


tft = TFTModel(
    input_chunk_length=60,
    output_chunk_length=1,
    hidden_size=40,
    lstm_layers=2,
    num_attention_heads=4,
    dropout=0.3,
    add_relative_index=True,
    pl_trainer_kwargs={
        "accelerator": "gpu",
        "devices": [0],
    },
)

tft.fit(
    series=y_train,
    past_covariates=X_train,
    verbose=False
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=100` reached.


TFTModel(output_chunk_shift=0, hidden_size=40, lstm_layers=2, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.3, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=True, skip_interpolation=False, loss_fn=None, likelihood=None, norm_type=LayerNorm, use_static_covariates=True, input_chunk_length=60, output_chunk_length=1, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': [0]})

### Prediction

In [66]:
train_start = y_train.time_index[tft.input_chunk_length]

y_pred_train = tft.historical_forecasts(
    series=y_train,
    past_covariates=X_train,
    start=train_start,
    forecast_horizon=1,
    stride=1,
    retrain=False,
    last_points_only=True,
    verbose=False,
)

y_pred_test = tft.historical_forecasts(
    series=y_test,
    past_covariates=X_test,
    forecast_horizon=1,     # predict 1 step ahead each time
    stride=1,               # every time point
    retrain=False,          # use your already-fitted model
    last_points_only=True,  # we only want the 1-step-ahead point
    verbose=False,
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


### Metrics

In [68]:
from darts.metrics import r2_score

metrics['TFT'] = {
        'Training_Accuracy': r2_score(y_train, y_pred_train) * 100 ,
        'Testing_Accuracy' : r2_score(y_test, y_pred_test) * 100
        }

print(f"Training accuracy: {metrics['TFT']['Training_Accuracy']:.3f}%")
print(f"Testing accuracy: {metrics['TFT']['Testing_Accuracy']:.3f}%")


Training accuracy: 55.265%
Testing accuracy: 48.812%


### Feature Engineering

Since the TCN was trained on scaled values, the predictions must be inverse-transformed to restore their original real-world meaning.

In [69]:
y_train = target_scaler.inverse_transform(y_train)
y_test = target_scaler.inverse_transform(y_test)
y_pred_train = target_scaler.inverse_transform(y_pred_train)
y_pred_test = target_scaler.inverse_transform(y_pred_test)

#### TFT Prediction + Residual

conbining the train + test prediction result back in to one

In [70]:
df_tft = pd.DataFrame(pd.concat([y_pred_train.to_dataframe(), y_pred_test.to_dataframe()], axis=0))
df_tft = df_tft.rename(columns={'price_change' : 'tft_pred'})
df_tft['tft_actual'] = pd.concat([y_train.to_dataframe(), y_test.to_dataframe()], axis=0)
df_tft['tft_res'] = df_tft['tft_actual'] - df_tft['tft_pred']
df_tft.head()

,tft_pred,tft_actual,tft_res
date,,,
2016-09-15,1.106662,-0.791855,-1.898517
2016-09-16,0.466457,-1.933283,-2.399740
2016-09-19,-0.587819,-0.613732,-0.025913
2016-09-20,-2.741241,-0.421941,2.319301
2016-09-21,-2.446118,0.345357,2.791475


#### Add TFT predictions as a new feature

In [71]:
df = pd.concat([df, df_tft[['tft_pred', 'tft_res']]], axis=1, join='inner')
df = df[[*[x for x in df.columns if x != target_col], target_col]]
df.head()

,norm_close,norm_obv_diff,macd_line,macd_diff,macd_signal,rsi,ev_ebidta,norm_vix_change,arima_pred,tcn_pred,tft_pred,tft_res,price_change
date,,,,,,,,,,,,,
2016-09-15,1.071299,0.445694,0.311392,0.100706,0.210686,71.660518,45.529253,2.207261,1.049647,-0.572279,1.106662,-1.898517,-0.791855
2016-09-16,1.062767,-0.449401,0.403155,0.153975,0.249180,69.458257,45.529253,-0.874227,-0.732588,0.108999,0.466457,-2.399740,-1.933283
2016-09-19,1.048230,-0.224992,0.445725,0.157236,0.288489,64.715657,45.529253,0.172703,-1.554167,-1.109635,-0.587819,-0.025913,-0.613732
2016-09-20,1.045770,0.161820,0.473999,0.148408,0.325591,64.715657,45.529253,-0.755341,-0.399364,-0.401494,-2.741241,2.319301,-0.421941
2016-09-21,1.043109,-0.170553,0.489951,0.131488,0.358463,64.550765,45.529253,-1.857969,-0.322163,-0.383086,-2.446118,2.791475,0.345357


#### Snapshot

In [ ]:
df.to_csv("/content/aapl_model_tft_dataset.csv", index=True, header=True)
#TO RUN locally
# df.to_csv("../data/aapl_model_tft_dataset.csv", index=True, header=True)

# Final Step

## XGB
XGBoost combines the predictions from TCN and TFT with the original features to produce the final enhanced forecast.

### Test Train Split

Proper time-series train/test split.  
10% of the latest time-series data will be used as test set.

In [72]:
target_col = 'tft_res'
og_col = 'price_change'
total_length = df.count().max()
split_len = int(total_length * 0.1)
split_idx = total_length - split_len
X, y = df[[x for x in df.columns if x != target_col and x != og_col]], df[[target_col]]
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

### Training

In [73]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",     # GPU trainer
    predictor="gpu_predictor",  # GPU inference,
    device="cuda",
    random_state=42,
)

xgb.fit(X_train, y_train)

/home/john-desktop/miniconda3/envs/rapids-new/lib/python3.10/site-packages/xgboost/training.py:199: UserWarning: [22:35:08] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,'cuda'
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


### Prediction

In [50]:
resid_hat_train = xgb.predict(X_train)
y_hat_train = X_train['tft_pred'] + resid_hat_train

resid_hat_test = xgb.predict(X_test)
y_hat_test = X_test['tft_pred'] + resid_hat_test


### Metrics

In [51]:
from sklearn.metrics import r2_score

metrics['XGB'] = {
        'Training_Accuracy': r2_score(df.iloc[:X_train.index.size]['price_change'], y_hat_train) * 100 ,
        'Testing_Accuracy' : r2_score(df.iloc[X_train.index.size:]['price_change'], y_hat_test) * 100
        }

print(f'Training accuracy: { metrics['XGB']['Training_Accuracy']:.3f}%')
print(f'Testing accuracy: { metrics['XGB']['Testing_Accuracy']:.3f}%')

Training accuracy: 97.793%
Testing accuracy: 66.617%


## Non Hybrid Linear Model

In [52]:
import pandas as pd
df = pd.DataFrame(pd.read_csv("/content/aapl_model_dataset.csv"))
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None).dt.normalize()
df.set_index('date', inplace=True)

### Train Test Split

In [53]:
target_col = 'price_change'
total_length = df.count().max()
split_len = int(total_length * 0.1)
split_idx = total_length - split_len
X, y = df[[x for x in df.columns if x != target_col]], df[[target_col]]
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

### Column Transform

In [54]:
from sklearn.preprocessing import StandardScaler

feature_scaler = StandardScaler()
X_train = feature_scaler.fit_transform(X_train)
X_test = feature_scaler.transform(X_test)

target_scaler = StandardScaler()
y_train = target_scaler.fit_transform(y_train)
y_test = target_scaler.transform(y_test)

### Training

In [55]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

### Metrics

In [56]:
metrics['Linear_Regression'] = {
        'Training_Accuracy': lr.score(X_train, y_train) * 100,
        'Testing_Accuracy' : lr.score(X_test, y_test) * 100
        }

print(f'Training accuracy: {metrics['Linear_Regression']['Training_Accuracy']:.3f}%')
print(f'Testing accuracy: {metrics['Linear_Regression']['Testing_Accuracy']:.3f}%')

Training accuracy: 0.571%
Testing accuracy: -0.481%


### Metrics Snapshot

In [57]:
df_metrics = pd.DataFrame.from_dict(metrics)
df_metrics.to_csv("/content/aapl_model_metrics.csv", index=True, header=True)

# Resources

* [https://www.kaggle.com/code/yasirabdaali/make-pandas-super-fast-with-gpu-acceleration](https://www.kaggle.com/code/yasirabdaali/make-pandas-super-fast-with-gpu-acceleration)
* [https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html](https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html)
* [https://pandas.pydata.org/docs/reference/index.html](https://pandas.pydata.org/docs/reference/index.html)
* [https://medium.com/rapids-ai/arima-forecast-large-time-series-datasets-with-rapids-cuml-18428a00d02e](https://medium.com/rapids-ai/arima-forecast-large-time-series-datasets-with-rapids-cuml-18428a00d02e)
* [https://unit8co.github.io/darts/index.html](https://unit8co.github.io/darts/index.html)